In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

Next we need to import the kwik file to analyze spike

In [3]:
hdf5_file_name = 'openephys.kwik'
data_set_name = '/channel_groups/0/spikes/clusters/main/1/cluster_group' #where is located the data you want inside the hdf5 file tree

file    = h5py.File(hdf5_file_name, 'r')   # 'r' means that hdf5 file is open in read-only mode
#data = file[data_set_name]

# see file tree
def print_attrs(name, obj):
    print name
    for key, val in obj.attrs.iteritems():
        print "    %s: %s" % (key, val)

file.visititems(print_attrs)


application_data
application_data/klustakwik2
    num_starting_clusters: 200
    max_possible_clusters: 400
application_data/spikedetekt
    threshold_strong_std_factor: 6
    use_single_threshold: 0
    filter_low: 500.0
    dtype: ['int16']
    threshold_weak_std_factor: 3.0
    n_channels: 32
    filter_lfp_low: 0
    raw_data_files: ['openephys.dat']
    chunk_size_seconds: 1
    filter_high_factor: 0.475
    prb_file: ['1x32_buzsaki']
    voltage_gain: 10.0
    extract_s_after: 16
    detect_spikes: ['negative']
    n_excerpts: 50
    filter_butter_order: 3
    weight_power: 2
    excerpt_size_seconds: 1
    connected_component_join_size: 1
    extract_s_before: 16
    experiment_name: ['openephys.kwik']
    sample_rate: 30000
    n_features_per_channel: 3
    chunk_overlap_seconds: 0.015
    pca_n_waveforms_max: 10000
    filter_lfp_high: 300
channel_groups
channel_groups/0
    channel_order: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28

channel_groups/0/clusters/main/182/application_data
channel_groups/0/clusters/main/182/application_data/klustaviewa
    color: 10
channel_groups/0/clusters/main/183
    cluster_group: 0
channel_groups/0/clusters/main/183/application_data
channel_groups/0/clusters/main/183/application_data/klustaviewa
    color: 4
channel_groups/0/clusters/main/184
    cluster_group: 0
channel_groups/0/clusters/main/184/application_data
channel_groups/0/clusters/main/184/application_data/klustaviewa
    color: 2
channel_groups/0/clusters/main/185
    cluster_group: 2
channel_groups/0/clusters/main/185/application_data
channel_groups/0/clusters/main/185/application_data/klustaviewa
    color: 7
channel_groups/0/clusters/main/186
    cluster_group: 0
channel_groups/0/clusters/main/186/application_data
channel_groups/0/clusters/main/186/application_data/klustaviewa
    color: 9
channel_groups/0/clusters/main/187
    cluster_group: 0
channel_groups/0/clusters/main/187/application_data
channel_groups/0/clust

    color: 8
channel_groups/0/clusters/original/120
    cluster_group: 3
channel_groups/0/clusters/original/120/application_data
channel_groups/0/clusters/original/120/application_data/klustaviewa
    color: 10
channel_groups/0/clusters/original/121
    cluster_group: 3
channel_groups/0/clusters/original/121/application_data
channel_groups/0/clusters/original/121/application_data/klustaviewa
    color: 2
channel_groups/0/clusters/original/122
    cluster_group: 3
channel_groups/0/clusters/original/122/application_data
channel_groups/0/clusters/original/122/application_data/klustaviewa
    color: 5
channel_groups/0/clusters/original/123
    cluster_group: 3
channel_groups/0/clusters/original/123/application_data
channel_groups/0/clusters/original/123/application_data/klustaviewa
    color: 5
channel_groups/0/clusters/original/124
    cluster_group: 3
channel_groups/0/clusters/original/124/application_data
channel_groups/0/clusters/original/124/application_data/klustaviewa
    color: 5
c

In [4]:
auxGood = file['/channel_groups/0/clusters/main/']
#a.visititems(print_attrs)
#a.attrs['cluster_group']

c = 0;
tag = np.zeros(np.size(np.concatenate((range(1),range(2,np.size(auxGood)+1)))))
for cls in np.concatenate((range(1),range(2,np.size(auxGood)+1))):
    aux = file['/channel_groups/0/clusters/main/' + str(cls)]
    tag[c] = aux.attrs['cluster_group'] #cluster tag as described below for 1 and 2
    c = c+1
#1 is MUA (needs to be cleaned in a manual way, we'll have to build an interface to do it)
#2 is GOOD clusters, just extract and analyze

#/channel_groups/0/clusters/main/<cluster_number>/cluster_group
aux = np.where(tag >= 1); #need to sum 1 to retrieve correct clusters

GoodMUAclus = aux[0]+1;
GoodMUAclus

array([  2,   3,   4,   5,   6,   7,   8,  10,  12,  14,  16,  17,  18,
        19,  25,  30,  44,  50,  51,  60,  61,  67, 107, 108, 109, 128,
       132, 142, 147, 153, 154, 155, 161, 162, 176, 178, 181, 185, 191, 201])

In [5]:
auxCluster = file['/channel_groups/0/spikes/clusters/main']
#auxChannel.visititems(print_attrs)
# '/channel_groups/0/spikes/time_samples' address to get sample time for each spike
# '/channel_groups/0/spikes/clusters/main' addres to get cluster for each spike
# 
np.shape(auxCluster)
#auxChannel.attrs['features_masks']

(4690147,)

In [6]:
hdf5_file_name = 'openephys.kwx'
data_set_name = '/channel_groups/0/spikes/clusters/main/1/cluster_group' #where is located the data you want inside the hdf5 file tree

arquiv    = h5py.File(hdf5_file_name, 'r')   # 'r' means that hdf5 file is open in read-only mode
selectChannel = arquiv['/channel_groups/0/features_masks']
np.shape(selectChannel)
#selectChannel[23,:,1] #the second dimension is indexing channel first and features laters, as it was said in the google groups, but some exploring would be good


(4690147, 96, 2)

Separating to which channel a spike should be

In [7]:
#Changin arrays to numpy arrays to work better
auxCluster = np.array(auxCluster)
GoodMUAclus = np.array(GoodMUAclus)

#create a dictionare to save the spikes for each clsuter
from collections import defaultdict
Neurons = defaultdict(list)

for cls in range(np.size(GoodMUAclus)):
    Neurons[GoodMUAclus[cls]]
    
for idx in range(np.size(GoodMUAclus)):
        aux = np.where(auxCluster == GoodMUAclus[idx]) #selecting which spike is in good clusters
        Neurons[GoodMUAclus[idx]] = aux[0]
        
        #save in a python structure which firing goes to each clsuter
        #if Neurons[aux2[0]] == []:
        #    Neurons[aux2[0]] = np.array(idx)
        #else:
        #    Neurons[aux2[0]] = np.array([Neurons[aux2[0]],idx])
        
Neurons

defaultdict(list,
            {2: array([   7749,   10349,   10462, ..., 4614497, 4620412, 4620699]),
             3: array([    244,     365,     390, ..., 4689945, 4690028, 4690086]),
             4: array([   4594,    7114,    7596, ..., 4689705, 4689983, 4690066]),
             5: array([     14,      73,     187, ..., 4690069, 4690085, 4690118]),
             6: array([  29082,   31656,   37987, ..., 4684095, 4684202, 4688494]),
             7: array([  11440,   15842,   16204, ..., 4687088, 4689562, 4689752]),
             8: array([     13,      35,     266, ..., 4689976, 4690001, 4690058]),
             10: array([     25,      30,      34, ..., 4689793, 4689794, 4689922]),
             12: array([      6,      16,      33, ..., 4689665, 4689681, 4689682]),
             14: array([   5911,   13109,   13120, ..., 4689888, 4689935, 4690004]),
             16: array([   5825,   11826,   12845, ..., 4689896, 4690063, 4690080]),
             17: array([ 199860,  229018,  450104, ...

Now I should use Neurons dict to get timestamps and channel. I should change the key from dict to the channel value, not cluster number

In [8]:
auxTimeStamps = file['/channel_groups/0/spikes/time_samples'] #getting the time sample for the time stamps
NeuronTS = defaultdict(list)
AllTS = range(np.size(NeuronTS))
for cls in range(np.size(GoodMUAclus)):
    aux = Neurons[GoodMUAclus[cls]][:]
    auxArr = np.zeros(np.size(aux))#array auxiliar
    for idx in range(np.size(aux)):
        auxArr[idx] = auxTimeStamps[aux[idx]]
    NeuronTS[GoodMUAclus[cls]] = auxArr
    
NeuronTS

defaultdict(list,
            {2: array([  4.61819000e+05,   6.03842000e+05,   6.17410000e+05, ...,
                      4.32750093e+08,   4.33470975e+08,   4.33512053e+08]),
             3: array([  1.95660000e+04,   3.03590000e+04,   3.21440000e+04, ...,
                      4.41734257e+08,   4.41741136e+08,   4.41749055e+08]),
             4: array([  2.14845000e+05,   3.95956000e+05,   4.46337000e+05, ...,
                      4.41698322e+08,   4.41738595e+08,   4.41746021e+08]),
             5: array([  1.02600000e+03,   7.03800000e+03,   1.62180000e+04, ...,
                      4.41747061e+08,   4.41748940e+08,   4.41751582e+08]),
             6: array([  2.25297100e+06,   2.55994900e+06,   3.14261300e+06, ...,
                      4.41144300e+08,   4.41157603e+08,   4.41589623e+08]),
             7: array([  6.84935000e+05,   1.08274400e+06,   1.12255000e+06, ...,
                      4.41465971e+08,   4.41678700e+08,   4.41706116e+08]),
             8: array([  1.0140000

For next we should save the channel to which one pertains and save the dict to use for analysis

In [9]:
Channels = np.zeros(np.size(GoodMUAclus))
for cls in range(np.size(GoodMUAclus)):
    aux = Neurons[GoodMUAclus[cls]][:]
    auxArr = np.zeros(np.size(aux))#array auxiliar
    #temp = np.zeros(32)
    for idx in range(np.size(aux)):
        for ch in range(32):
            temp = np.sum(selectChannel[aux[idx],(3*ch):(3*ch)+3,1]) #three features per channel
            if (temp == 3):
                auxArr[idx] = ch
                break
    auxCh = stats.mode(auxArr)
    Channels[cls] = auxCh[0][0]
    
Channels
    
#selectChannel[spikenumber,feature,1]

array([ 17.,   6.,  16.,  10.,  19.,  21.,  19.,  28.,  15.,  20.,  11.,
         6.,  27.,  19.,  30.,  27.,   3.,   6.,  29.,   6.,  29.,  22.,
         6.,  31.,  27.,   6.,  28.,   6.,  27.,   6.,  29.,   6.,  20.,
        28.,  19.,   6.,  27.,  30.,  28.,  27.])

In [10]:
NeuronTS['Channels'] = Channels
NeuronTS['Cluster'] = GoodMUAclus

np.save('NeuronTS.npy', NeuronTS)
